In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    image.flags.writeable = False # image is no longer writeable
    results = model.process(image) # make prediction
    image.flags.writeable = True # image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw right hand connections
    

In [5]:
def draw_styled_landmarks(image, results):
    # draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow("OpenCV Feed", image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [9]:
frame.shape

(480, 640, 3)

In [7]:
image.shape 

(480, 640, 3)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('train_data')

# Actions that we try to detect
actions = np.array(os.listdir(DATA_PATH))

# Thirty videos worth of data
no_sequences = 40

# Videos are going to be 30 frames in length
sequence_length = 40

In [63]:
actions

array(['angry', 'cat', 'cold', 'dog', 'good', 'happy', 'Hello', 'hot',
       'hungry', 'Im full', 'me', 'no', 'sad', 'sorry', 'Thank you',
       'tired', 'we', 'why', 'worry', 'you'], dtype='<U9')

In [9]:
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'angry': 0,
 'cat': 1,
 'cold': 2,
 'dog': 3,
 'good': 4,
 'happy': 5,
 'Hello': 6,
 'hot': 7,
 'hungry': 8,
 'Im full': 9,
 'me': 10,
 'no': 11,
 'sad': 12,
 'sorry': 13,
 'Thank you': 14,
 'tired': 15,
 'we': 16,
 'why': 17,
 'worry': 18,
 'you': 19}

In [24]:
videos = {action:[] for action in actions}

In [25]:
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num+1)))
            window.append(res)
        videos[action].append(window)

In [12]:
import pickle

In [32]:
with open('videos_dic.pkl', 'wb') as tf:
    pickle.dump(videos, tf)

In [13]:
with open('videos.pkl', 'rb') as tf:
    videos = pickle.load(tf)

In [14]:
with open('sequences.pkl', 'rb') as tf:
    sequences = pickle.load(tf)

In [12]:
np.array(videos["angry"]).shape

(40, 40, 1662)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# 단어별

In [123]:
for action in videos.keys():
    X = np.array(videos[action])
    labels = [0 for _ in range(len(actions))]
    labels[label_map[action]] = 1
    y = np.array([labels for _ in range(no_sequences)])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
    model = Sequential()
    model.add(LSTM(32, return_sequences=True, activation='sigmoid', input_shape=(sequence_length, 1662)))
    model.add(LSTM(64, return_sequences=True, activation='sigmoid'))
    model.add(LSTM(32, return_sequences=False, activation='sigmoid'))
    model.add(Dense(32, activation='sigmoid'))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.fit(X_train, y_train, epochs=2000)
    
    model.save("models/{}.h5".format(action))

Epoch 1/2000
2/2 [==============================] - 9s 142ms/step - loss: 2.9558 - categorical_accuracy: 0.0000e+00
Epoch 2/2000
2/2 [==============================] - 0s 139ms/step - loss: 3.6329 - categorical_accuracy: 0.0556
Epoch 3/2000
2/2 [==============================] - 0s 133ms/step - loss: 7.7066 - categorical_accuracy: 0.8333
Epoch 4/2000
2/2 [==============================] - 0s 134ms/step - loss: 8.5752 - categorical_accuracy: 0.8056
Epoch 5/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.9247 - categorical_accuracy: 0.8889
Epoch 6/2000
2/2 [==============================] - 0s 135ms/step - loss: 5.4260 - categorical_accuracy: 0.8611
Epoch 7/2000
2/2 [==============================] - 0s 126ms/step - loss: 4.4502 - categorical_accuracy: 0.9167
Epoch 8/2000
2/2 [==============================] - 0s 133ms/step - loss: 1.6094 - categorical_accuracy: 0.9167
Epoch 9/2000
2/2 [==============================] - 0s 138ms/step - loss: 0.4288 - categorical_accur

2/2 [==============================] - 0s 138ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 141/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 142/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 143/2000
2/2 [==============================] - 0s 128ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 144/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 145/2000
2/2 [==============================] - 0s 133ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 146/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 147/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 148/2000
2/2 [==============================] - 0s 136ms/step - l

2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 279/2000
2/2 [==============================] - 0s 132ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 280/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 281/2000
2/2 [==============================] - 0s 136ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 282/2000
2/2 [==============================] - 0s 137ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 283/2000
2/2 [==============================] - 0s 136ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 284/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 285/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 286/2000
2/2 [==============================] - 0s 130ms/step - l

2/2 [==============================] - 0s 129ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 417/2000
2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 418/2000
2/2 [==============================] - 0s 137ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 419/2000
2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 420/2000
2/2 [==============================] - 0s 133ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 421/2000
2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 422/2000
2/2 [==============================] - 0s 132ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 423/2000
2/2 [==============================] - 0s 136ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 424/2000
2/2 [==============================] - 0s 130ms/step - l

2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 555/2000
2/2 [==============================] - 0s 146ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 556/2000
2/2 [==============================] - 0s 145ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 557/2000
2/2 [==============================] - 0s 143ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 558/2000
2/2 [==============================] - 0s 140ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 559/2000
2/2 [==============================] - 0s 140ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 560/2000
2/2 [==============================] - 0s 129ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 561/2000
2/2 [==============================] - 0s 137ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 562/2000
2/2 [==============================] - 0s 139ms/step - l

...

Epoch 1370/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1371/2000
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1372/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1373/2000
2/2 [==============================] - 0s 135ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1374/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1375/2000
2/2 [==============================] - 0s 138ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1376/2000
2/2 [==============================] - 0s 132ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1377/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1378/2000
2/2 [===========================

Epoch 1507/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1508/2000
2/2 [==============================] - 0s 134ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1509/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1510/2000
2/2 [==============================] - 0s 123ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1511/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1512/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1513/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1514/2000
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1515/2000
2/2 [===========================

Epoch 1644/2000
2/2 [==============================] - 0s 124ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1645/2000
2/2 [==============================] - 0s 124ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1646/2000
2/2 [==============================] - 0s 122ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1647/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1648/2000
2/2 [==============================] - 0s 129ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1649/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1650/2000
2/2 [==============================] - 0s 128ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1651/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1652/2000
2/2 [===========================

Epoch 1781/2000
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1782/2000
2/2 [==============================] - 0s 130ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1783/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1784/2000
2/2 [==============================] - 0s 133ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1785/2000
2/2 [==============================] - 0s 128ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1786/2000
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1787/2000
2/2 [==============================] - 0s 128ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1788/2000
2/2 [==============================] - 0s 126ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1789/2000
2/2 [===========================

Epoch 1918/2000
2/2 [==============================] - 0s 129ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1919/2000
2/2 [==============================] - 0s 127ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1920/2000
2/2 [==============================] - 0s 129ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1921/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1922/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1923/2000
2/2 [==============================] - 0s 135ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1924/2000
2/2 [==============================] - 0s 131ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1925/2000
2/2 [==============================] - 0s 125ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 1926/2000
2/2 [===========================

In [38]:
action = "angry"

X = np.array(videos[action])
labels = [0 for _ in range(len(actions))]
labels[label_map[action]] = 1
y = np.array([labels for _ in range(no_sequences)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='sigmoid', input_shape=(sequence_length, 1662)))
model.add(LSTM(64, return_sequences=True, activation='sigmoid'))
model.add(LSTM(32, return_sequences=False, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
model.fit(X_train, y_train, epochs=2000)

Epoch 1/2000
2/2 [==============================] - 9s 91ms/step - loss: 4.7017 - categorical_accuracy: 0.0000e+00
Epoch 2/2000
2/2 [==============================] - 0s 99ms/step - loss: 4.6225 - categorical_accuracy: 0.0000e+00
Epoch 3/2000
2/2 [==============================] - 0s 84ms/step - loss: 4.5447 - categorical_accuracy: 0.0000e+00
Epoch 4/2000
2/2 [==============================] - 0s 79ms/step - loss: 4.4684 - categorical_accuracy: 0.0000e+00
Epoch 5/2000
2/2 [==============================] - 0s 106ms/step - loss: 4.3936 - categorical_accuracy: 0.0000e+00
Epoch 6/2000
2/2 [==============================] - 0s 82ms/step - loss: 4.3201 - categorical_accuracy: 0.0000e+00
Epoch 7/2000
2/2 [==============================] - 0s 81ms/step - loss: 4.2479 - categorical_accuracy: 0.0000e+00
Epoch 8/2000
2/2 [==============================] - 0s 81ms/step - loss: 4.1771 - categorical_accuracy: 0.0000e+00
Epoch 9/2000
2/2 [==============================] - 0s 76ms/step - loss: 4.1077

2/2 [==============================] - 0s 97ms/step - loss: 0.2779 - categorical_accuracy: 1.0000
Epoch 145/2000
2/2 [==============================] - 0s 91ms/step - loss: 0.2723 - categorical_accuracy: 1.0000
Epoch 146/2000
2/2 [==============================] - 0s 97ms/step - loss: 0.2669 - categorical_accuracy: 1.0000
Epoch 147/2000
2/2 [==============================] - 0s 99ms/step - loss: 0.2616 - categorical_accuracy: 1.0000
Epoch 148/2000
2/2 [==============================] - 0s 82ms/step - loss: 0.2565 - categorical_accuracy: 1.0000
Epoch 149/2000
2/2 [==============================] - 0s 76ms/step - loss: 0.2514 - categorical_accuracy: 1.0000
Epoch 150/2000
2/2 [==============================] - 0s 78ms/step - loss: 0.2466 - categorical_accuracy: 1.0000
Epoch 151/2000
2/2 [==============================] - 0s 76ms/step - loss: 0.2418 - categorical_accuracy: 1.0000
Epoch 152/2000
2/2 [==============================] - 0s 80ms/step - loss: 0.2372 - categorical_accuracy: 1.000

2/2 [==============================] - 0s 81ms/step - loss: 0.0505 - categorical_accuracy: 1.0000
Epoch 289/2000
2/2 [==============================] - 0s 81ms/step - loss: 0.0501 - categorical_accuracy: 1.0000
Epoch 290/2000
2/2 [==============================] - 0s 83ms/step - loss: 0.0498 - categorical_accuracy: 1.0000
Epoch 291/2000
2/2 [==============================] - 0s 77ms/step - loss: 0.0494 - categorical_accuracy: 1.0000
Epoch 292/2000
2/2 [==============================] - 0s 78ms/step - loss: 0.0491 - categorical_accuracy: 1.0000
Epoch 293/2000
2/2 [==============================] - 0s 76ms/step - loss: 0.0488 - categorical_accuracy: 1.0000
Epoch 294/2000
2/2 [==============================] - 0s 83ms/step - loss: 0.0484 - categorical_accuracy: 1.0000
Epoch 295/2000
2/2 [==============================] - 0s 80ms/step - loss: 0.0481 - categorical_accuracy: 1.0000
Epoch 296/2000
2/2 [==============================] - 0s 82ms/step - loss: 0.0478 - categorical_accuracy: 1.000

KeyboardInterrupt: 

In [40]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
accuracy_score(ytrue, yhat)

1/1 [==============================] - 1s 864ms/step


1.0

# 데이터 구분

In [160]:
X_train = np.expand_dims(videos["angry"][0], axis=0)
y_train = np.zeros((1, 20),dtype=np.float32)

for key in videos.keys():
    cat = np.expand_dims(np.array([0 for _ in range(len(actions))]), axis=0)
    cat[0][label_map[key]] = 1
    
    for i in range(38):
        X_train = np.append(X_train, np.expand_dims(videos[key][i], axis=0), axis=0)
        y_train = np.append(y_train, cat, axis=0)
        
X_train = np.delete(X_train, 0, axis=0)
y_train = np.delete(y_train, 0, axis=0)

In [161]:
X_test = np.expand_dims(videos["angry"][0], axis=0)
y_test = np.zeros((1, 20), dtype=np.float32)

for key in videos.keys():
    cat = np.expand_dims(np.array([0 for _ in range(len(actions))]), axis=0)
    cat[0][label_map[key]] = 1
    
    X_test = np.append(X_test, np.expand_dims(videos[key][38], axis=0), axis=0)
    y_test = np.append(y_test, cat, axis=0)
    
X_test = np.delete(X_test, 0, axis=0)
y_test = np.delete(y_test, 0, axis=0)

In [162]:
X_val = np.expand_dims(videos["angry"][0], axis=0)
y_val = np.zeros((1, 20), dtype=np.float32)
i = 0
for key in videos.keys():
    cat = np.expand_dims(np.array([0 for _ in range(len(actions))]), axis=0)
    cat[0][label_map[key]] = 1
    
    X_val = np.append(X_val, np.expand_dims(videos[key][39], axis=0), axis=0)
    y_val = np.append(y_val, cat, axis=0)
    
X_val = np.delete(X_val, 0, axis=0)
y_val = np.delete(y_val, 0, axis=0)

In [146]:
with open('dataset/X_train.pkl', 'wb') as tf:
    pickle.dump(X_train, tf)

In [147]:
with open('dataset/y_train.pkl', 'wb') as tf:
    pickle.dump(y_train, tf)

In [148]:
with open('dataset/X_test.pkl', 'wb') as tf:
    pickle.dump(X_test, tf)

In [149]:
with open('dataset/y_test.pkl', 'wb') as tf:
    pickle.dump(y_test, tf)

In [150]:
with open('dataset/X_val.pkl', 'wb') as tf:
    pickle.dump(X_val, tf)

In [151]:
with open('dataset/y_val.pkl', 'wb') as tf:
    pickle.dump(y_val, tf)

In [16]:
with open('dataset/X_train.pkl', 'rb') as tf:
    X_train = pickle.load(tf)
with open('dataset/X_test.pkl', 'rb') as tf:
    X_test = pickle.load(tf)
with open('dataset/X_val.pkl', 'rb') as tf:
    X_val = pickle.load(tf)
with open('dataset/y_train.pkl', 'rb') as tf:
    y_train = pickle.load(tf)
with open('dataset/y_test.pkl', 'rb') as tf:
    y_test = pickle.load(tf)
with open('dataset/y_val.pkl', 'rb') as tf:
    y_val = pickle.load(tf)

# 전체

In [17]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)

In [43]:
# sequences, labels = [], []
# for action in actions:
#     for sequence in range(no_sequences):
#         window = []
#         for frame_num in range(sequence_length):
#             res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num+1)))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])

In [44]:
with open('sequences.pkl', 'wb') as tf:
    pickle.dump(sequences, tf)

In [ ]:
with open('videos_dic.pkl', 'rb') as tf:
    sequences = pickle.load(tf)

In [45]:
# X = np.array(sequences)
# y = to_categorical(labels).astype(int)

In [157]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

(760, 40, 1662)

In [16]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='sigmoid', input_shape=(sequence_length, 1662)))
model.add(LSTM(64, return_sequences=True, activation='sigmoid'))
model.add(LSTM(32, return_sequences=False, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [167]:
model.fit(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), shuffle=True)

Epoch 1/1000
24/24 [==============================] - 17s 459ms/step - loss: 3.0887 - categorical_accuracy: 0.0500 - val_loss: 3.0483 - val_categorical_accuracy: 0.0500
Epoch 2/1000
24/24 [==============================] - 10s 432ms/step - loss: 3.0344 - categorical_accuracy: 0.0500 - val_loss: 3.0166 - val_categorical_accuracy: 0.0500
Epoch 3/1000
24/24 [==============================] - 10s 422ms/step - loss: 3.0127 - categorical_accuracy: 0.0500 - val_loss: 3.0046 - val_categorical_accuracy: 0.0500
Epoch 4/1000
24/24 [==============================] - 10s 431ms/step - loss: 3.0031 - categorical_accuracy: 0.0500 - val_loss: 2.9932 - val_categorical_accuracy: 0.0500
Epoch 5/1000
24/24 [==============================] - 9s 367ms/step - loss: 2.9888 - categorical_accuracy: 0.0763 - val_loss: 2.9719 - val_categorical_accuracy: 0.1000
Epoch 6/1000
24/24 [==============================] - 10s 399ms/step - loss: 2.9691 - categorical_accuracy: 0.0829 - val_loss: 2.9441 - val_categorical_accu

24/24 [==============================] - 9s 396ms/step - loss: 1.9160 - categorical_accuracy: 0.2697 - val_loss: 1.8331 - val_categorical_accuracy: 0.2000
Epoch 98/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.8785 - categorical_accuracy: 0.3053 - val_loss: 1.8085 - val_categorical_accuracy: 0.3000
Epoch 99/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.8720 - categorical_accuracy: 0.3303 - val_loss: 1.8071 - val_categorical_accuracy: 0.2500
Epoch 100/1000
24/24 [==============================] - 10s 399ms/step - loss: 1.8710 - categorical_accuracy: 0.3184 - val_loss: 1.8117 - val_categorical_accuracy: 0.2500
Epoch 101/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.8758 - categorical_accuracy: 0.3132 - val_loss: 1.7749 - val_categorical_accuracy: 0.3500
Epoch 102/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.8591 - categorical_accuracy: 0.3132 - val_loss: 1.7951 - val_categorical_accuracy:

24/24 [==============================] - 9s 397ms/step - loss: 1.5109 - categorical_accuracy: 0.3645 - val_loss: 1.4151 - val_categorical_accuracy: 0.3500
Epoch 194/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.4802 - categorical_accuracy: 0.3855 - val_loss: 1.4889 - val_categorical_accuracy: 0.4000
Epoch 195/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.4525 - categorical_accuracy: 0.3697 - val_loss: 1.3548 - val_categorical_accuracy: 0.3500
Epoch 196/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.4083 - categorical_accuracy: 0.3974 - val_loss: 1.3564 - val_categorical_accuracy: 0.4500
Epoch 197/1000
24/24 [==============================] - 9s 393ms/step - loss: 1.3991 - categorical_accuracy: 0.3882 - val_loss: 1.3420 - val_categorical_accuracy: 0.4500
Epoch 198/1000
24/24 [==============================] - 9s 386ms/step - loss: 1.3910 - categorical_accuracy: 0.4079 - val_loss: 1.3877 - val_categorical_accuracy:

24/24 [==============================] - 10s 397ms/step - loss: 1.3081 - categorical_accuracy: 0.4500 - val_loss: 1.2578 - val_categorical_accuracy: 0.4500
Epoch 242/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.3141 - categorical_accuracy: 0.4395 - val_loss: 1.2202 - val_categorical_accuracy: 0.6000
Epoch 243/1000
24/24 [==============================] - 10s 397ms/step - loss: 1.2739 - categorical_accuracy: 0.4474 - val_loss: 1.2251 - val_categorical_accuracy: 0.6000
Epoch 244/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.2748 - categorical_accuracy: 0.4447 - val_loss: 1.2357 - val_categorical_accuracy: 0.5500
Epoch 245/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.2808 - categorical_accuracy: 0.4921 - val_loss: 1.2166 - val_categorical_accuracy: 0.5500
Epoch 246/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.2765 - categorical_accuracy: 0.4276 - val_loss: 1.2246 - val_categorical_accura

24/24 [==============================] - 10s 399ms/step - loss: 1.1706 - categorical_accuracy: 0.5408 - val_loss: 1.1639 - val_categorical_accuracy: 0.4500
Epoch 290/1000
24/24 [==============================] - 10s 399ms/step - loss: 1.1737 - categorical_accuracy: 0.5132 - val_loss: 1.1527 - val_categorical_accuracy: 0.5000
Epoch 291/1000
24/24 [==============================] - 10s 412ms/step - loss: 1.1659 - categorical_accuracy: 0.5618 - val_loss: 1.1906 - val_categorical_accuracy: 0.4000
Epoch 292/1000
24/24 [==============================] - 10s 398ms/step - loss: 1.1608 - categorical_accuracy: 0.5197 - val_loss: 1.1416 - val_categorical_accuracy: 0.5000
Epoch 293/1000
24/24 [==============================] - 9s 390ms/step - loss: 1.1447 - categorical_accuracy: 0.5329 - val_loss: 1.1694 - val_categorical_accuracy: 0.6500
Epoch 294/1000
24/24 [==============================] - 9s 381ms/step - loss: 1.1445 - categorical_accuracy: 0.5474 - val_loss: 1.1402 - val_categorical_accuracy

...

24/24 [==============================] - 11s 451ms/step - loss: 0.9679 - categorical_accuracy: 0.6461 - val_loss: 1.0762 - val_categorical_accuracy: 0.5000
Epoch 722/1000
24/24 [==============================] - 11s 445ms/step - loss: 0.9258 - categorical_accuracy: 0.6487 - val_loss: 0.6680 - val_categorical_accuracy: 0.6500
Epoch 723/1000
24/24 [==============================] - 11s 452ms/step - loss: 1.7436 - categorical_accuracy: 0.5211 - val_loss: 1.5220 - val_categorical_accuracy: 0.5500
Epoch 724/1000
24/24 [==============================] - 11s 439ms/step - loss: 1.0497 - categorical_accuracy: 0.6039 - val_loss: 0.7177 - val_categorical_accuracy: 0.7000
Epoch 725/1000
24/24 [==============================] - 11s 448ms/step - loss: 0.7042 - categorical_accuracy: 0.7118 - val_loss: 0.5418 - val_categorical_accuracy: 0.6500
Epoch 726/1000
24/24 [==============================] - 11s 454ms/step - loss: 0.5977 - categorical_accuracy: 0.7513 - val_loss: 0.6167 - val_categorical_accura

24/24 [==============================] - 11s 460ms/step - loss: 0.4503 - categorical_accuracy: 0.7974 - val_loss: 0.4621 - val_categorical_accuracy: 0.7500
Epoch 770/1000
24/24 [==============================] - 11s 456ms/step - loss: 0.4506 - categorical_accuracy: 0.7934 - val_loss: 0.4701 - val_categorical_accuracy: 0.7500
Epoch 771/1000
24/24 [==============================] - 11s 452ms/step - loss: 0.4490 - categorical_accuracy: 0.7974 - val_loss: 0.4775 - val_categorical_accuracy: 0.7000
Epoch 772/1000
24/24 [==============================] - 11s 448ms/step - loss: 0.4500 - categorical_accuracy: 0.8066 - val_loss: 0.4677 - val_categorical_accuracy: 0.7500
Epoch 773/1000
24/24 [==============================] - 11s 447ms/step - loss: 0.4477 - categorical_accuracy: 0.7882 - val_loss: 0.4744 - val_categorical_accuracy: 0.7500
Epoch 774/1000
24/24 [==============================] - 11s 445ms/step - loss: 0.4470 - categorical_accuracy: 0.7961 - val_loss: 0.4768 - val_categorical_accura

Epoch 817/1000
24/24 [==============================] - 9s 371ms/step - loss: 0.4364 - categorical_accuracy: 0.7829 - val_loss: 0.4665 - val_categorical_accuracy: 0.7000
Epoch 818/1000
24/24 [==============================] - 9s 395ms/step - loss: 0.4363 - categorical_accuracy: 0.7684 - val_loss: 0.4767 - val_categorical_accuracy: 0.7000
Epoch 819/1000
24/24 [==============================] - 11s 460ms/step - loss: 0.4355 - categorical_accuracy: 0.7921 - val_loss: 0.4603 - val_categorical_accuracy: 0.7500
Epoch 820/1000
24/24 [==============================] - 11s 444ms/step - loss: 0.4350 - categorical_accuracy: 0.8039 - val_loss: 0.4741 - val_categorical_accuracy: 0.7500
Epoch 821/1000
24/24 [==============================] - 7s 306ms/step - loss: 0.4337 - categorical_accuracy: 0.8053 - val_loss: 0.4745 - val_categorical_accuracy: 0.7000
Epoch 822/1000
24/24 [==============================] - 10s 409ms/step - loss: 0.4348 - categorical_accuracy: 0.7974 - val_loss: 0.4734 - val_catego

24/24 [==============================] - 7s 291ms/step - loss: 0.4723 - categorical_accuracy: 0.8026 - val_loss: 0.4504 - val_categorical_accuracy: 0.7500
Epoch 866/1000
24/24 [==============================] - 7s 296ms/step - loss: 0.4685 - categorical_accuracy: 0.8079 - val_loss: 0.4773 - val_categorical_accuracy: 0.7500
Epoch 867/1000
24/24 [==============================] - 8s 329ms/step - loss: 0.4830 - categorical_accuracy: 0.8013 - val_loss: 0.4510 - val_categorical_accuracy: 0.7000
Epoch 868/1000
24/24 [==============================] - 7s 305ms/step - loss: 0.4689 - categorical_accuracy: 0.8105 - val_loss: 0.4563 - val_categorical_accuracy: 0.7000
Epoch 869/1000
24/24 [==============================] - 7s 280ms/step - loss: 0.4766 - categorical_accuracy: 0.8039 - val_loss: 0.4448 - val_categorical_accuracy: 0.7000
Epoch 870/1000
24/24 [==============================] - 8s 329ms/step - loss: 0.4715 - categorical_accuracy: 0.7987 - val_loss: 0.4384 - val_categorical_accuracy: 0.

24/24 [==============================] - 11s 448ms/step - loss: 0.3662 - categorical_accuracy: 0.8539 - val_loss: 0.3545 - val_categorical_accuracy: 0.8500
Epoch 962/1000
24/24 [==============================] - 11s 442ms/step - loss: 0.3669 - categorical_accuracy: 0.8487 - val_loss: 0.3488 - val_categorical_accuracy: 0.8500
Epoch 963/1000
24/24 [==============================] - 11s 460ms/step - loss: 0.3627 - categorical_accuracy: 0.8579 - val_loss: 0.3461 - val_categorical_accuracy: 0.8500
Epoch 964/1000
24/24 [==============================] - 11s 443ms/step - loss: 0.3624 - categorical_accuracy: 0.8434 - val_loss: 0.3459 - val_categorical_accuracy: 0.8500
Epoch 965/1000
24/24 [==============================] - 11s 445ms/step - loss: 0.3637 - categorical_accuracy: 0.8605 - val_loss: 0.3385 - val_categorical_accuracy: 0.8500
Epoch 966/1000
24/24 [==============================] - 11s 458ms/step - loss: 0.3635 - categorical_accuracy: 0.8474 - val_loss: 0.3442 - val_categorical_accura

In [23]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [24]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
accuracy_score(ytrue, yhat)

1/1 [==============================] - 0s 72ms/step


0.8

In [169]:
model.save("{}.h5".format("second_trial"))

In [26]:
model.load_weights('second_trial.h5')

# 테스트

In [17]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [18]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
    image.flags.writeable = False # image is no longer writeable
    results = model.process(image) # make prediction
    image.flags.writeable = True # image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
    return image, results

In [19]:
def draw_styled_landmarks(image, results):
    # draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

In [20]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [21]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*20), (int(prob*100), 90+num*20), colors[0], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

- you(80)
- worry(c:2 s:97)
- why(0.02)
- we(96)
- tired(c:0.1 s:97)
- thank you(1)
- sorry(95)
- sad(0.00001)
- no(0.03)
- me(0.1)
- Im full(43)
- hungry(47)
- hot(h:95 h:0.000001)
- Hello(0.002)
- happy(0.08)
- good(0.1)
- dog(97)
- cold(95)
- cat(h:60 h:70)
- angry(0.006)

# GOOD(11) - WIN!!!
### you we sorry dog cold cat worry hot tired hungry Im full

# BAD(9)
## why thankyou sad no me hello happy good angry

In [29]:
# 1. New detection variables
sequence = []
res = np.zeros((20)) 
threshold = 0.7
action = "dog"

cap = cv2.VideoCapture(cv2.CAP_DSHOW)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-40:]
        
        if len(sequence) == 40:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)], res[np.argmax(res)])
            
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
#         cv2.putText(image, "{} {}".format(actions[np.argmax(res)], res[np.argmax(res)]*100), (3, 30),
#                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, "{} {}".format(action, res[label_map[action]]*100), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow("OpenCV Feed", image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step
me 0.88422334
1/1 [==============================] - 0s 19ms/step
me 0.8696677
1/1 [==============================] - 0s 23ms/step
me 0.7623894
1/1 [==============================] - 0s 20ms/step
me 0.5256066
1/1 [==============================] - 0s 25ms/step
tired 0.5829869
1/1 [==============================] - 0s 23ms/step
tired 0.6945969
1/1 [==============================] - 0s 22ms/step
tired 0.7537633
1/1 [==============================] - 0s 23ms/step
tired 0.7882414
1/1 [==============================] - 0s 22ms/step
tired 0.8341623
1/1 [==============================] - 0s 23ms/step
tired 0.86549014
1/1 [==============================] - 0s 19ms/step
tired 0.88375753
1/1 [==============================] - 0s 20ms/step
tired 0.89469
1/1 [==============================] - 0s 21ms/step
tired 0.90324795
1/1 [==============================] - 0s 21ms/step
tired 0.91239035
1/1 [==============================] - 0s 23ms/step
tired

In [24]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
res = np.zeros((20)) 
threshold = 0.7

cap = cv2.VideoCapture(cv2.CAP_DSHOW)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-40:]
        
        if len(sequence) == 40:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)], res[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        # 3. Viz Logic
        if len(predictions) >= 10:
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                    
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow("OpenCV Feed", image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
sorry 0.097573265
1/1 [==============================] - 0s 18ms/step
sorry 0.09757317
1/1 [==============================] - 0s 22ms/step
sorry 0.097573206
1/1 [==============================] - 0s 20ms/step
sorry 0.09757325
1/1 [==============================] - 0s 18ms/step
sorry 0.09757301
1/1 [==============================] - 0s 20ms/step
sorry 0.097573034
1/1 [==============================] - 0s 37ms/step
sorry 0.09757314
1/1 [==============================] - 0s 20ms/step
sorry 0.09757316
1/1 [==============================] - 0s 20ms/step
sorry 0.09757299
1/1 [==============================] - 0s 20ms/step
sorry 0.09757317
1/1 [==============================] - 0s 18ms/step
sorry 0.09757291
1/1 [==============================] - 0s 23ms/step
sorry 0.097572885
1/1 [==============================] - 0s 21ms/step
sorry 0.09757324
1/1 [==============================] - 0s 20ms/step
sorry 0.09757345
1/1 [=========================

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])